# About

It's difficult to use functions in jupyter notebook, since we want different steps to be in different cells, so one of the main functions of this module is to emulate a function like scope of the variables - which get destroyed at the end of the experiment. Some extra magic is added to reclaim GPU and General RAM.

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from ipyexperiments import *

## Setup and preload

In [3]:
import numpy as np
import torch

In [4]:
def consume_gpu(n): return torch.ones((n, n)).cuda()
def consume_cpu(n): return np.ones((n, n))

`pytorch`'s CUDA machinery seems to require ~0.5GB GPU RAM, and ~2GB of RAM upon its first use, and it's not shared between processes. So if you use pytorch w/ CUDA your GPU card is 0.5GB smaller from the get going, and multiply that by the number of concurrent processes. 

Because of that, in order to get the numbers right, let's pre-load it by allocating a tiny tensor on `cuda`. If we don't - the first experiments' stats will be misleading/incorrect.

In [5]:
z = consume_gpu(1)

## Experiment 1: Consume general RAM

Let's consume a big chunk of non-GPU RAM and reclaim it at the end of the experiment

In [6]:
exp1 = IPyExperiments() # consume some cpu ram

Starting experiment...

*** Current state:
Gen RAM Free 15.0 GB | Proc size 2.2 GB
GPU RAM Free  8.0 GB | Used 516.9 MB | Util 6.1% | Total 8.5 GB




In [7]:
x1 = consume_cpu(2**14) # about 1.5GB

In [8]:
x2 = consume_cpu(2**14) # about 1.5GB

In [9]:
del exp1 # finish experiment

Finishing experiment...

*** Deleting the following local variables:
['x1', 'x2']

*** RAM consumed during the experiment:
Gen: 4.3 GB
GPU: 0 Bytes

*** RAM reclaimed at the end of the experiment:
Gen: 4.3 GB (100.00%)
GPU: 0 Bytes (100.00%)

*** Elapsed wallclock time:
00:00:01

*** Current state:
Gen RAM Free 14.9 GB | Proc size 2.2 GB
GPU RAM Free  8.0 GB | Used 516.9 MB | Util 6.1% | Total 8.5 GB




## Experiment 2: Consume general and GPU RAM

Let's consume a big chunk of non-GPU and GPU RAM and reclaim both at the end of the experiment

In [10]:
exp2 = IPyExperiments() # consume some gpu and cpu ram

Starting experiment...

*** Current state:
Gen RAM Free 14.9 GB | Proc size 2.2 GB
GPU RAM Free  8.0 GB | Used 516.9 MB | Util 6.1% | Total 8.5 GB




In [11]:
x1 = consume_cpu(2**14) # about 1.5GB

In [12]:
x2 = consume_gpu(2**14) # about 1GB

In [13]:
del exp2 # finish experiment

Finishing experiment...

*** Deleting the following local variables:
['x1', 'x2']

*** RAM consumed during the experiment:
Gen: 2.1 GB
GPU: 1.1 GB

*** RAM reclaimed at the end of the experiment:
Gen: 2.1 GB (100.00%)
GPU: 1.1 GB (100.00%)

*** Elapsed wallclock time:
00:00:01

*** Current state:
Gen RAM Free 14.9 GB | Proc size 2.2 GB
GPU RAM Free  8.0 GB | Used 516.9 MB | Util 6.1% | Total 8.5 GB




## Experiment 3: Get stats data, preserve some vars

Here we demonstate features that help with using this framework programmatically. i.e. getting the functions to return stats during and at the end of the experiment, rather than just printing it. You can then use it to programmatically refine the parameters before rerunning the experiment.

This experiment also demonstrates how to save some of the local variables.

In [14]:
exp3 = IPyExperiments() # consume some gpu and cpu ram

Starting experiment...

*** Current state:
Gen RAM Free 14.9 GB | Proc size 2.2 GB
GPU RAM Free  8.0 GB | Used 516.9 MB | Util 6.1% | Total 8.5 GB




In [15]:
x1 = consume_cpu(2**14) # about 1.5GB

Run an intermediary report of how much of the resources was consumed, and how much is available, returning the data as numbers. (none would be reclaimed yet, so it'll be zeros, but the return value is there for consistency).

In [16]:
consumed, reclaimed, available = exp3.get_stats()
print(consumed, reclaimed, available)

{'gen_ram': 2147504128, 'gpu_ram': 0} {'gen_ram': 0, 'gpu_ram': 0} {'gen_ram': 12794216448, 'gpu_ram': 7996440576}


Let's preserve these variables, so that they remain available after the experiment is finished and the rest of the local variables get deleted. 

Note, that you need to pass the names of the variables and not the variables themselves.

In [17]:
exp3.keep_var_names('consumed', 'reclaimed', 'available')

In [18]:
x2 = consume_gpu(2**14) # about 1GB

Run another intermediary report.

In [19]:
consumed, reclaimed, available = exp3.get_stats()
print(consumed, reclaimed, available)

{'gen_ram': 2147504128, 'gpu_ram': 1073741824} {'gen_ram': 0, 'gpu_ram': 0} {'gen_ram': 12793061376, 'gpu_ram': 6922698752}


Complete the experiment, delete local vars, reclaim memory, and run the final report of how much of the resources was consumed, and how much is available, and how much was reclaimed, returning the data as numbers.

In [20]:
final_consumed, final_reclaimed, final_available = exp3.finish() # finish experiment
print("\nNumerical data:\n", final_consumed, final_reclaimed, final_available)

Finishing experiment...

*** Deleting the following local variables:
['x1', 'exp3', 'x2']

*** Keeping the following local variables:
['consumed', 'reclaimed', 'available']

*** RAM consumed during the experiment:
Gen: 2.1 GB
GPU: 1.1 GB

*** RAM reclaimed at the end of the experiment:
Gen: 2.1 GB (100.00%)
GPU: 1.1 GB (100.00%)

*** Elapsed wallclock time:
00:00:01

*** Current state:
Gen RAM Free 14.9 GB | Proc size 2.2 GB
GPU RAM Free  8.0 GB | Used 516.9 MB | Util 6.1% | Total 8.5 GB



Numerical data:
 {'gen_ram': 2147504128, 'gpu_ram': 1073741824} {'gen_ram': 2147487744, 'gpu_ram': 1073741824} {'gen_ram': 14942650368, 'gpu_ram': 7996440576}


And let's test that we can still access the variables we asked not to destroy:

In [21]:
print("\nHalf-way data:\n", consumed, reclaimed, available)


Half-way data:
 {'gen_ram': 2147504128, 'gpu_ram': 1073741824} {'gen_ram': 0, 'gpu_ram': 0} {'gen_ram': 12793061376, 'gpu_ram': 6922698752}


## Experiment 4: Context manager

If you want to put all cells into one, you could simplify the experiment even further by using its context manager.

In [22]:
with IPyExperiments(): 
    x1 = consume_cpu(2**14)
    x2 = consume_gpu(2**14)

Starting experiment...

*** Current state:
Gen RAM Free 14.9 GB | Proc size 2.2 GB
GPU RAM Free  8.0 GB | Used 516.9 MB | Util 6.1% | Total 8.5 GB


Finishing experiment...

*** Deleting the following local variables:
['x1', 'x2']

*** RAM consumed during the experiment:
Gen: 2.1 GB
GPU: 1.1 GB

*** RAM reclaimed at the end of the experiment:
Gen: 2.1 GB (100.00%)
GPU: 1.1 GB (100.00%)

*** Elapsed wallclock time:
00:00:01

*** Current state:
Gen RAM Free 14.9 GB | Proc size 2.2 GB
GPU RAM Free  8.0 GB | Used 516.9 MB | Util 6.1% | Total 8.5 GB




In [23]:
with IPyExperiments() as exp: 
    x1 = consume_cpu(2**14)
    z = "some data"
    x2 = consume_gpu(2**14)
    exp.keep_var_names('z')
print(z)

Starting experiment...

*** Current state:
Gen RAM Free 14.9 GB | Proc size 2.2 GB
GPU RAM Free  8.0 GB | Used 516.9 MB | Util 6.1% | Total 8.5 GB


Finishing experiment...

*** Deleting the following local variables:
['exp', 'x1', 'x2']

*** Keeping the following local variables:
['z']

*** RAM consumed during the experiment:
Gen: 2.1 GB
GPU: 1.1 GB

*** RAM reclaimed at the end of the experiment:
Gen: 2.1 GB (100.00%)
GPU: 1.1 GB (100.20%)

*** Elapsed wallclock time:
00:00:01

*** Current state:
Gen RAM Free 14.9 GB | Proc size 2.2 GB
GPU RAM Free  8.0 GB | Used 514.9 MB | Util 6.0% | Total 8.5 GB


some data


In [24]:
# prevent committing an unsaved notebook
%%javascript
IPython.notebook.save_notebook()

<IPython.core.display.Javascript object>